# Pinecone

In [1]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [10]:
%pip install pinecone -q

Note: you may need to restart the kernel to use updated packages.


In [12]:
from pinecone import Pinecone
pinecone = Pinecone()

pinecone.list_indexes()

[]

## Working with Pinecone Indexes

In [13]:
pinecone.list_indexes().names()

[]

In [33]:
from pinecone import ServerlessSpec
index_name = 'langchain'

if index_name not in pinecone.list_indexes().names():
  print(f'Creating index: {index_name}')
  pinecone.create_index(
    name=index_name,
    dimension=1536,
    metric='cosine',
    spec=ServerlessSpec(
      cloud='aws',
      region='us-east-1',
    )
  )
  print(f'Index {index_name} created')

else:
  print(f'Index {index_name} already exists')

Creating index: langchain
Index langchain created


In [31]:
index_name = 'langchain'
if index_name in pinecone.list_indexes().names():
  print(f'Deleting index: {index_name}')
  pinecone.delete_index(name=index_name)
  print(f'Index {index_name} deleted')
else:
  print(f'Index {index_name} does not exist')

Deleting index: langchain
Index langchain deleted


In [34]:
index = pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

## Working with Vectors

In [35]:
import random
vectors = [
  [random.random() for _ in range(1536)] for _ in range(10)
]
ids = list('abcde')
index_name = 'langchain'
index = pinecone.Index(index_name)

index.upsert(
  vectors=zip(ids, vectors)
)

{'upserted_count': 5}

In [24]:
# updating vectors
index.upsert(
  vectors=[('c', [0.5]*1536)]
)

{'upserted_count': 1}

In [ ]:
# fetching vectors
index.fetch(ids=['c', 'd'])

In [27]:
# deleting vectors
index.delete(ids=['b', 'c'])

{}

In [36]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 5}},
 'total_vector_count': 5,
 'vector_type': 'dense'}

In [30]:
index.fetch(ids=['x'])

FetchResponse(namespace='', vectors={}, usage={'read_units': 1})

In [37]:
# query
query_vector = [random.random() for _ in range(1536)]

In [38]:
index.query(
  vector=query_vector,
  top_k=3,
  include_values=False
)

{'matches': [{'id': 'b', 'score': 0.761747718, 'values': []},
             {'id': 'e', 'score': 0.748847365, 'values': []},
             {'id': 'a', 'score': 0.74758333, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 1}}

### Namespaces

In [42]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

In [ ]:
import random

index = pinecone.Index(index_name)
ids = list('abcde')
index_name = 'langchain'

vectors = [
  [random.random() for _ in range(1536)] for _ in range(5)
]
index.upsert(
  vectors=zip(ids, vectors)
)

{'upserted_count': 5}

In [44]:
vectors = [
  [random.random() for _ in range(1536)] for _ in range(3)
]

In [ ]:
ids = list('xyz')
index.upsert(
  vectors=zip(ids, vectors),
  namespace='first-namespace'
)

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 5}},
 'total_vector_count': 5,
 'vector_type': 'dense'}

In [49]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 5},
                'first-namespace': {'vector_count': 3}},
 'total_vector_count': 8,
 'vector_type': 'dense'}

In [ ]:
index.fetch(ids=['x'], namespace='first-namespace')

In [52]:
index.delete(ids=['x'], namespace='first-namespace')

{}

In [54]:
index.delete(delete_all=True, namespace='first-namespace')

{}

In [55]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 5}},
 'total_vector_count': 5,
 'vector_type': 'dense'}

{}